In [395]:
import pandas as pd
import numpy as np
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,8)

from itertools import combinations
import researchpy as rp

from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import OneHotEncoder 

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = None 

In [396]:
df = pd.read_pickle('../data/cardio_risk.pkl')
df.head()

,education,sex,age_category,BMI_category,is_smoking,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,2.0,F,Senior Citizens,Overweight,YES,0.0,0,0,0,64.0,3.0,221.0,148.0,85.0,27.62,90.0,80.0,1
1,4.0,M,Middle-Aged Adults,Overweight,NO,0.0,0,1,0,36.0,0.0,212.0,168.0,98.0,29.77,72.0,75.0,0
2,1.0,F,Middle-Aged Adults,Normal Weight,YES,0.0,0,0,0,46.0,10.0,250.0,116.0,71.0,20.35,88.0,94.0,0
3,1.0,M,Middle-Aged Adults,Overweight,YES,0.0,0,1,0,50.0,20.0,233.0,158.0,88.0,28.26,68.0,94.0,1
4,1.0,F,Senior Citizens,Overweight,YES,0.0,0,0,0,64.0,30.0,241.0,136.5,85.0,26.42,70.0,77.0,0


In [397]:
df.drop(['diaBP'], axis = 1, inplace = True)

In [398]:
patients = {'education': [1.0, 3.0, 2.0, 4.0, 2.0, 4.0, 3.0, 1.0, 1.0, 3.0],
        'sex': ['F', 'M', 'M', 'F', 'M', 'F', 'M', 'F', 'F', 'M'],
        'age_category': ['Senior Citizens', 'Middle-Aged Adults', 'Young Adults', 'Senior Citizens', 'Middle-Aged Adults', 'Senior Citizens', 'Middle-Aged Adults', 'Young Adults', 'Middle-Aged Adults', 'Senior Citizens'],
        'BMI_category': ['Obesity', 'Overweight', 'Normal Weight', 'Overweight', 'Normal Weight', 'Obesity', 'Overweight', 'Normal Weight', 'Normal Weight', 'Obesity'],
        'is_smoking': ['YES', 'NO', 'YES', 'NO', 'NO', 'YES', 'NO', 'YES', 'YES', 'NO'],
        'BPMeds': [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0],
        'prevalentStroke': [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        'prevalentHyp': [1, 0, 1, 0, 0, 1, 1, 0, 1, 0],
        'diabetes': [1, 1, 0, 0, 0, 1, 0, 1, 0, 0],
        'age': [63, 48, 32, 61, 39, 69, 55, 34, 52, 75],
        'cigsPerDay': [40.0, 0.0, 23.0, 0.0, 0.0, 25.0, 0.0, 15.0, 30.0, 0.0],
        'totChol': [282.0, 165.0, 130.0, 199.0, 160.0, 220.0, 178.0, 110.0, 190.0, 170.0],
        'sysBP': [158.0, 124.5, 140.0, 127.0, 110.0, 149.0, 152.0, 119.0, 150.0, 116.0], 
        'BMI': [41, 27.5, 22.5, 28, 24, 34, 29.0, 23.0, 24.5, 31.0],
        'heartRate': [105.0, 89, 79, 92, 76, 94, 110, 75, 91, 77],
        'glucose': [192.0, 232.0, 110, 92, 82, 156.0, 94.0, 130.0, 116.0, 89.0]}

In [399]:
patients_df = pd.DataFrame (patients)
patients_df 

,education,sex,age_category,BMI_category,is_smoking,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose
0,1.0,F,Senior Citizens,Obesity,YES,1.0,1,1,1,63,40.0,282.0,158.0,41.0,105.0,192.0
1,3.0,M,Middle-Aged Adults,Overweight,NO,0.0,0,0,1,48,0.0,165.0,124.5,27.5,89.0,232.0
2,2.0,M,Young Adults,Normal Weight,YES,1.0,0,1,0,32,23.0,130.0,140.0,22.5,79.0,110.0
3,4.0,F,Senior Citizens,Overweight,NO,0.0,0,0,0,61,0.0,199.0,127.0,28.0,92.0,92.0
4,2.0,M,Middle-Aged Adults,Normal Weight,NO,0.0,0,0,0,39,0.0,160.0,110.0,24.0,76.0,82.0
5,4.0,F,Senior Citizens,Obesity,YES,1.0,0,1,1,69,25.0,220.0,149.0,34.0,94.0,156.0
6,3.0,M,Middle-Aged Adults,Overweight,NO,0.0,1,1,0,55,0.0,178.0,152.0,29.0,110.0,94.0
7,1.0,F,Young Adults,Normal Weight,YES,0.0,0,0,1,34,15.0,110.0,119.0,23.0,75.0,130.0
8,1.0,F,Middle-Aged Adults,Normal Weight,YES,1.0,0,1,0,52,30.0,190.0,150.0,24.5,91.0,116.0
9,3.0,M,Senior Citizens,Obesity,NO,0.0,0,0,0,75,0.0,170.0,116.0,31.0,77.0,89.0


In [400]:
categorical_list = ['education', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'sex', 'is_smoking']

for col in categorical_list:
    patients_df [col] = patients_df [col].astype('category')

In [401]:
patients_df ['BMI_category'] = pd.cut(df['BMI'], bins=[0, 18.5, 24.9, 29.9, float('inf')], labels=['Underweight', 'Normal Weight', 'Overweight', 'Obesity'])
patients_df ['age_category'] = pd.cut(df['age'], bins=[0, 17, 35, 59, float('inf')], labels=['Minors', 'Young Adults', 'Middle-Aged Adults', 'Senior Citizens'])

In [402]:
new_order_columns = ['education', 'sex', 'age_category', 'BMI_category', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes',
                    'age', 'cigsPerDay', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']

In [403]:
patients_df  = patients_df [new_order_columns]

In [404]:
patients_df_num = patients_df.select_dtypes(include = np.number)
patients_df_cat = patients_df.select_dtypes(include = ['category'])

##### SCALING

In [405]:
with open('../data/robust_scaler.pkl', mode = "rb") as robust_scaler:
    scaler = pickle.load(robust_scaler)

In [406]:
num_scaler = pd.DataFrame(scaler.transform(patients_df_num), columns = patients_df_num.columns)
num_scaler

,age,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose
0,1.000000,2.00,0.827586,1.092593,3.144439,2.000000,8.142857
1,-0.071429,0.00,-1.189655,-0.148148,0.426774,0.933333,11.000000
2,-1.214286,1.15,-1.793103,0.425926,-0.579768,0.266667,2.285714
3,0.857143,0.00,-0.603448,-0.055556,0.527428,1.133333,1.000000
4,-0.714286,0.00,-1.275862,-0.685185,-0.277806,0.066667,0.285714
5,1.428571,1.25,-0.241379,0.759259,1.735279,1.266667,5.571429
6,0.428571,0.00,-0.965517,0.870370,0.728737,2.333333,1.142857
7,-1.071429,0.75,-2.137931,-0.351852,-0.479114,0.000000,3.714286
8,0.214286,1.50,-0.758621,0.796296,-0.177151,1.066667,2.714286
9,1.857143,0.00,-1.103448,-0.462963,1.131354,0.133333,0.785714


In [407]:
patients_df[num_scaler.columns] = num_scaler

##### ENCODING

CATEGORIES WITHOUT A SPECIFIC ORDER

In [408]:
with open('../data/onehotencoder.pkl', mode = "rb") as onehotencoder:
    oh_encoding = pickle.load(onehotencoder)

In [409]:
patients_df

,education,sex,age_category,BMI_category,is_smoking,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose
0,1.0,F,Senior Citizens,Overweight,YES,1.0,1,1,1,1.000000,2.00,0.827586,1.092593,3.144439,2.000000,8.142857
1,3.0,M,Middle-Aged Adults,Overweight,NO,0.0,0,0,1,-0.071429,0.00,-1.189655,-0.148148,0.426774,0.933333,11.000000
2,2.0,M,Middle-Aged Adults,Normal Weight,YES,1.0,0,1,0,-1.214286,1.15,-1.793103,0.425926,-0.579768,0.266667,2.285714
3,4.0,F,Middle-Aged Adults,Overweight,NO,0.0,0,0,0,0.857143,0.00,-0.603448,-0.055556,0.527428,1.133333,1.000000
4,2.0,M,Senior Citizens,Overweight,NO,0.0,0,0,0,-0.714286,0.00,-1.275862,-0.685185,-0.277806,0.066667,0.285714
5,4.0,F,Senior Citizens,Obesity,YES,1.0,0,1,1,1.428571,1.25,-0.241379,0.759259,1.735279,1.266667,5.571429
6,3.0,M,Senior Citizens,Normal Weight,NO,0.0,1,1,0,0.428571,0.00,-0.965517,0.870370,0.728737,2.333333,1.142857
7,1.0,F,Middle-Aged Adults,Overweight,YES,0.0,0,0,1,-1.071429,0.75,-2.137931,-0.351852,-0.479114,0.000000,3.714286
8,1.0,F,Middle-Aged Adults,Normal Weight,YES,1.0,0,1,0,0.214286,1.50,-0.758621,0.796296,-0.177151,1.066667,2.714286
9,3.0,M,Middle-Aged Adults,Overweight,NO,0.0,0,0,0,1.857143,0.00,-1.103448,-0.462963,1.131354,0.133333,0.785714


In [410]:
columns_without_order = ['education', 'sex', 'is_smoking']

In [411]:
def encoding_onehotencoder(dataframe, columns, oh):

    transform = oh.transform(dataframe[columns])
    oh_dataframe = pd.DataFrame(transform.toarray(), columns = oh.get_feature_names_out())

    dataframe = pd.concat([dataframe,oh_dataframe],axis=1)

    dataframe.drop(columns = columns, inplace = True)

    return dataframe

In [412]:
patients_df = encoding_onehotencoder(patients_df, columns_without_order, oh_encoding)

In [413]:
patients_df

,age_category,BMI_category,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose,education_1.0,education_2.0,education_3.0,education_4.0,sex_F,sex_M,is_smoking_NO,is_smoking_YES
0,Senior Citizens,Overweight,1.0,1,1,1,1.000000,2.00,0.827586,1.092593,3.144439,2.000000,8.142857,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,Middle-Aged Adults,Overweight,0.0,0,0,1,-0.071429,0.00,-1.189655,-0.148148,0.426774,0.933333,11.000000,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,Middle-Aged Adults,Normal Weight,1.0,0,1,0,-1.214286,1.15,-1.793103,0.425926,-0.579768,0.266667,2.285714,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,Middle-Aged Adults,Overweight,0.0,0,0,0,0.857143,0.00,-0.603448,-0.055556,0.527428,1.133333,1.000000,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,Senior Citizens,Overweight,0.0,0,0,0,-0.714286,0.00,-1.275862,-0.685185,-0.277806,0.066667,0.285714,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
5,Senior Citizens,Obesity,1.0,0,1,1,1.428571,1.25,-0.241379,0.759259,1.735279,1.266667,5.571429,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
6,Senior Citizens,Normal Weight,0.0,1,1,0,0.428571,0.00,-0.965517,0.870370,0.728737,2.333333,1.142857,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
7,Middle-Aged Adults,Overweight,0.0,0,0,1,-1.071429,0.75,-2.137931,-0.351852,-0.479114,0.000000,3.714286,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,Middle-Aged Adults,Normal Weight,1.0,0,1,0,0.214286,1.50,-0.758621,0.796296,-0.177151,1.066667,2.714286,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9,Middle-Aged Adults,Overweight,0.0,0,0,0,1.857143,0.00,-1.103448,-0.462963,1.131354,0.133333,0.785714,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


CATEGORIES WITH A SPECIFIC ORDER

In [414]:
dict_bpmeds = {0.0: 0, 1.0: 2}
dict_prevalentstroke = {0.0: 0, 1.0: 3}
dict_diabetes = {0.0: 0, 1.0: 2}
dict_bmi_category = {'Normal Weight': 0, 'Overweight': 0.25, 'Underweight': 0.5, 'Obesity': 0.5}
dict_age_category = {'Young Adults': 0, 'Middle-Aged Adults': 0.5, 'Senior Citizens': 2}

In [415]:
columns_map = ['BPMeds', 'prevalentStroke', 'diabetes', 'BMI_category', 'age_category']
maps_encoding = [dict_bpmeds, dict_prevalentstroke, dict_diabetes, dict_bmi_category, dict_age_category ]

In [416]:
def encoding_map(dataframe, maps, columns):

    for map_encoding, col in zip(maps, columns):
        dataframe[col] = dataframe[col].map(map_encoding)

    return dataframe

In [417]:
patients_df = encoding_map(patients_df, maps_encoding, columns_map)

In [418]:
patients_df

,age_category,BMI_category,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose,education_1.0,education_2.0,education_3.0,education_4.0,sex_F,sex_M,is_smoking_NO,is_smoking_YES
0,2.0,0.25,2,3,1,2,1.000000,2.00,0.827586,1.092593,3.144439,2.000000,8.142857,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.5,0.25,0,0,0,2,-0.071429,0.00,-1.189655,-0.148148,0.426774,0.933333,11.000000,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.5,0.00,2,0,1,0,-1.214286,1.15,-1.793103,0.425926,-0.579768,0.266667,2.285714,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.5,0.25,0,0,0,0,0.857143,0.00,-0.603448,-0.055556,0.527428,1.133333,1.000000,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,2.0,0.25,0,0,0,0,-0.714286,0.00,-1.275862,-0.685185,-0.277806,0.066667,0.285714,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
5,2.0,0.50,2,0,1,2,1.428571,1.25,-0.241379,0.759259,1.735279,1.266667,5.571429,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
6,2.0,0.00,0,3,1,0,0.428571,0.00,-0.965517,0.870370,0.728737,2.333333,1.142857,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
7,0.5,0.25,0,0,0,2,-1.071429,0.75,-2.137931,-0.351852,-0.479114,0.000000,3.714286,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
8,0.5,0.00,2,0,1,0,0.214286,1.50,-0.758621,0.796296,-0.177151,1.066667,2.714286,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9,0.5,0.25,0,0,0,0,1.857143,0.00,-1.103448,-0.462963,1.131354,0.133333,0.785714,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


#### PREDICTION WITH XG BOOST

In [419]:
with open ('../data/xg_boost_pred.pkl', 'rb') as xg_boost:
        xg_boost_pred = pickle.load(xg_boost)

In [420]:
list_results_xg_boost = list(xg_boost_pred.predict(patients_df))

In [421]:
list_results_xg_boost

[0.7322473,
 0.37191764,
 0.029101994,
 0.588131,
 0.011030384,
 0.69111985,
 0.32750174,
 0.32600868,
 0.67472607,
 0.12609918]

In [422]:
def convert_to_percentage(pred_list):

    for num in range(len(pred_list)):
        pred_list[num] = round(pred_list[num] * 100, 2)

    return pred_list

In [423]:
list_results_xg_boost = convert_to_percentage(list_results_xg_boost)
list_results_xg_boost

[73.22, 37.19, 2.91, 58.81, 1.1, 69.11, 32.75, 32.6, 67.47, 12.61]

In [448]:
def print_results_xg_boost(pred_list):
    
    for index, patient in enumerate(pred_list):

        print(f'The risk of coronary heart disease in Patient {index + 1} is {patient} %')

        if patient > 50:
            print(f'Prediction: YES, the Patient {index + 1} is at risk of a coronary heart disease.\n')
        else:
            print(f'Prediction: NO, the Patient {index + 1} is not at significant risk of a coronary heart disease.\n')

In [449]:
print_results_xg_boost(list_results_xg_boost)

The risk of coronary heart disease in Patient 1 is 73.22 %
Prediction: YES, the Patient 1 is at risk of a coronary heart disease.

The risk of coronary heart disease in Patient 2 is 37.19 %
Prediction: NO, the Patient 2 is not at significant risk of a coronary heart disease.

The risk of coronary heart disease in Patient 3 is 2.91 %
Prediction: NO, the Patient 3 is not at significant risk of a coronary heart disease.

The risk of coronary heart disease in Patient 4 is 58.81 %
Prediction: YES, the Patient 4 is at risk of a coronary heart disease.

The risk of coronary heart disease in Patient 5 is 1.1 %
Prediction: NO, the Patient 5 is not at significant risk of a coronary heart disease.

The risk of coronary heart disease in Patient 6 is 69.11 %
Prediction: YES, the Patient 6 is at risk of a coronary heart disease.

The risk of coronary heart disease in Patient 7 is 32.75 %
Prediction: NO, the Patient 7 is not at significant risk of a coronary heart disease.

The risk of coronary heart

#### PREDICTION WITH RANDOM FOREST

In [433]:
with open ('../data/random_forest_pred.pkl', 'rb') as rf:
        random_forest_pred = pickle.load(rf)

In [434]:
list_results_rf = list(random_forest_pred.predict(patients_df))

In [435]:
list_results_rf 

[1, 1, 0, 1, 0, 1, 1, 0, 1, 0]

In [436]:
def print_results_random_forest(pred_list):
    
    for index, patient in enumerate(pred_list):

        if patient > 0.5:
            print(f'Prediction: YES, the Patient {index + 1} is at risk of a coronary heart disease.\n')
        else:
            print(f'Prediction: NO, the Patient {index + 1} is not at significant risk of a coronary heart disease.\n')

In [437]:
print_results_random_forest(list_results_rf)

Prediction: YES, the Patient 1 is at risk of a coronary heart disease.

Prediction: YES, the Patient 2 is at risk of a coronary heart disease.

Prediction: NO, the Patient 3 is not at significant risk of a coronary heart disease.

Prediction: YES, the Patient 4 is at risk of a coronary heart disease.

Prediction: NO, the Patient 5 is not at significant risk of a coronary heart disease.

Prediction: YES, the Patient 6 is at risk of a coronary heart disease.

Prediction: YES, the Patient 7 is at risk of a coronary heart disease.

Prediction: NO, the Patient 8 is not at significant risk of a coronary heart disease.

Prediction: YES, the Patient 9 is at risk of a coronary heart disease.

Prediction: NO, the Patient 10 is not at significant risk of a coronary heart disease.

